In [1]:
using LightOSM
using Random
using BenchmarkTools
using LightOSM
using StatsBase
Random.seed!(1234);

## 下载地图数据
有三种方式：

- place
- bbox
- point

In [ ]:
# 下载一个区域
data = download_osm_network(:bbox; save_to_file_location="data/beijing.osm", minlat=39.8267, minlon=116.2636, maxlat=39.9908, maxlon=116.4903);

## 从地图数据构建对象
- graph_from_object 从对象
- graph_from_download 下载并创建
- graph_from_object 从文件

In [ ]:
# 从之前下载的对象
g = graph_from_object(data, weight_type=:distance);

In [3]:
# 从下载的文件
g = graph_from_file("data/beijing.osm", weight_type=:distance);

┌ Info: Created OSMGraph object with kwargs: network_type=drive, weight_type=distance, graph_type=static, precompute_dijkstra_states=false, largest_connected_component=true
└ @ LightOSM /home/chenniu/.julia/packages/LightOSM/jlfp9/src/graph.jl:47


## 获取节点

In [4]:
# 所有节点
nodes = keys(g.nodes) |> collect;

In [5]:
length(nodes)

44422

In [6]:
origs = sample(nodes, 1000)
dests = sample(nodes, 1000)
od = zip(origs, dests) |> collect;

In [7]:
n1 = g.nodes[1230361512]
n2 = g.nodes[5172728454];

In [8]:
### 节点经纬度

In [9]:
n1.location.lat, n1.location.lon

(39.8513049, 116.2799398)

### 计算节点之间的距离

In [10]:
distance(n1, n2) # km

2.292438979288037

### 计算角度

In [11]:
heading(n1, n2) # degrees from North

143.75721269536155

## 计算最短路径

In [12]:
function shortest_safe(g::OSMGraph, n1::Int64, n2::Int64)
    try
        shortest_path(g, n1, n2, algorithm=:astar)
    catch
        Int[]
    end
end

shortest_safe (generic function with 1 method)

In [14]:
route = shortest_safe(g, od[1]...)

115-element Array{Int64,1}:
   35835755
 1807791765
 3071671562
  244887119
  260726382
   30993647
 1807791797
 1807791800
 1807791805
   30993644
   66420639
   30993643
 1807791814
          ⋮
  288190967
 3060379044
  288191077
  528923583
  528923616
 1293134755
  321557747
 1293134766
 1771677052
  380731617
  288193610
 1771677049

In [15]:
@btime shortest_safe(g, od[1]...)

  1.709 ms (576 allocations: 543.00 KiB)


115-element Array{Int64,1}:
   35835755
 1807791765
 3071671562
  244887119
  260726382
   30993647
 1807791797
 1807791800
 1807791805
   30993644
   66420639
   30993643
 1807791814
          ⋮
  288190967
 3060379044
  288191077
  528923583
  528923616
 1293134755
  321557747
 1293134766
 1771677052
  380731617
  288193610
 1771677049

## 计算路径长度

In [16]:
function get_distance(g::OSMGraph, path::Array)
    weights = weights_from_path(g, path)
    cum_weights = cumsum(weights)
    total_distance = round(cum_weights[end], digits=2)
    total_distance
end

get_distance (generic function with 1 method)

In [17]:
get_distance(g, route) # km

19.53